The main focus of this notebook are surface waves and their dispersion. 
Using synthetic seismograms generated with Instaseis (http://instaseis.net/), we try to recover the dispersion curves of Love and Rayleigh waves in the PREM model.

Author: Thomas Ulrich

In [ ]:
import instaseis
import obspy
from obspy.taup.taup_geo import calc_dist
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pycwt as wavelet
import pandas as pd
from scipy import interpolate

# db = instaseis.open_db("syngine://prem_a_10s")
db = instaseis.open_db("syngine://prem_i_2s")

We first generate synthetic seismograms with Instaseis by specifying moment tensor, source and receiver locations.
The source and receivers locations are fully arbitrary (we align source and receivers along a meridian for commodity).

In [ ]:
# Source
lats = 89.91
lons = 74.4940
# receiver
latr = 42.6390
lonr = 74.4940

In [ ]:
receiver = instaseis.Receiver(latitude=latr, longitude=lonr, network="AB", station="CED")
t0 = obspy.UTCDateTime(2011, 1, 2, 3, 4, 5)
source = instaseis.Source(
    latitude=lats,
    longitude=lons,
    depth_in_m=12000,
    m_rr=4.710000e24 / 1e7,
    m_tt=3.810000e22 / 1e7,
    m_pp=-4.740000e24 / 1e7,
    m_rt=3.990000e23 / 1e7,
    m_rp=-8.050000e23 / 1e7,
    m_tp=-1.230000e24 / 1e7,
    origin_time=t0,
)
st = db.get_seismograms(
    source=source, receiver=receiver, components=["Z", "R", "T"], kind="displacement"
)
# convert to micro_m
for i in range(3):
    st[i].data *= 1e6
st.plot(show=False)

## Group Velocity

If dispersion occurs, waves of different wavelengths travel at different speeds.  
In the following, we band-pass filter the seismograms around various central periods, ranging from 5 to 240s.

In [ ]:
iZRT = 1
dt = st[iZRT].stats.delta
duration = st[iZRT].stats.npts * dt
myPeriods = np.logspace(np.log2(10.0 * dt), np.log2(duration / 15), 10, base=2)

nper = len(myPeriods)

figall, axarr = plt.subplots(nper + 1, 1, figsize=(14, 9), dpi=160, sharex=True, sharey=False)


ti = st[iZRT].times(reftime=t0)
axarr[0].plot(ti, st[iZRT].data, label="no filter")
axarr[0].legend(loc="upper left")

pickedArrival = np.zeros((nper,))
for i, T in enumerate(myPeriods):
    st_temp = st.copy()
    st_temp.filter("bandpass", freqmin=0.85 / T, freqmax=1.15 / T, corners=4, zerophase=True)
    axarr[i + 1].plot(
        ti,
        st_temp[iZRT].data,
        label=f"T={T:.1f}s",
    )
    # Envelope of filtered data
    data_envelope = obspy.signal.filter.envelope(st_temp[iZRT].data)
    axarr[i + 1].plot(ti, data_envelope.data)
    idmax = np.argmax(data_envelope.data)
    pickedArrival[i] = ti[idmax]
    axarr[i + 1].legend(loc="upper left")


axarr[0].set_xlim([0, 2200])

axarr[nper].set_xlabel("time (s)")
axarr[nper // 2].set_ylabel("displacement (um)")
plt.show()

d = {"filter_Tc": myPeriods, "picked_arrival": pickedArrival}
df = pd.DataFrame(data=d)
pd.options.display.float_format = "{:.0f}".format
print(df)

The idea of using a narrow filter to pick the arrival is imprecise and challenging with real data. Instead, a wavelet transform approach allows tracking more accurately the central frequency with time ("A wavelet is a wave-like oscillation with an amplitude that begins at zero, increases or decreases, and then returns to zero one or more times", https://en.wikipedia.org/wiki/Wavelet). Such a solution is implemented below (inspired from http://regeirk.github.io/pycwt/tutorial.html):

In [ ]:
def compute_wavelet_power_spectrum(mydata, dt):
    # we normalize the data
    std = mydata.std()
    dat_norm = mydata / std

    mother = wavelet.Morlet(6)

    s0 = 2 * dt  # Starting scale
    dj = 1 / 12.0  # Twelve sub-octaves per octaves
    J = 8.0 / dj  # Height powers of two with dj sub-octaves

    # wave is shaped (nfreq, nti), with nfreq = J + 1 = 8 * 12 +1,
    # and nti the number of samples of the signal
    wave, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(dat_norm, dt, dj, s0, J, mother)
    iwave = wavelet.icwt(wave, scales, dt, dj, mother) * std
    period = 1.0 / freqs

    # compute power spectrum
    power = (np.abs(wave)) ** 2
    # rectify the power spectrum according to the suggestions proposed by Liu et al. (2007)
    power /= scales[:, None]
    return period, power, wave, coi


periodsWT, power, wave, coi = compute_wavelet_power_spectrum(st[iZRT].data, dt)

# Plot wavelet power spectrum figure
figall, ax = plt.subplots()
levels = [16 / 2**i for i in range(10, -1, -1)]
ax.contourf(
    ti, np.log2(periodsWT), np.log2(power), np.log2(levels), extend="both", cmap=plt.cm.viridis
)

if False:
    # change to True to also plot the "cone of influence" as a hatched area
    ax.fill(
        np.concatenate([ti, ti[-1:] + dt, ti[-1:] + dt, ti[:1] - dt, ti[:1] - dt]),
        np.concatenate(
            [np.log2(coi), [1e-9], np.log2(periodsWT[-1:]), np.log2(periodsWT[-1:]), [1e-9]]
        ),
        "k",
        alpha=0.3,
        hatch="x",
    )


ax.set_title(f"Wavelet Power Spectrum")
ax.set_xlabel("time (s)")
ax.set_ylabel("period (s)")
ax.grid()
Yticks = 2 ** np.arange(np.ceil(np.log2(periodsWT.min())), np.ceil(np.log2(periodsWT.max())))

ax.set_yticks(np.log2(Yticks))
ax.set_yticklabels(Yticks)

plt.xlim(0, 2200)

In [ ]:
def pickArrivalWavelet(ti, power, period, tmin=1050, tmax=3000):
    # pick arrival time from the wavelet Power Spectrum data
    maxs = np.amax(power, axis=1)
    # enforce that the arrival time should be greater than tmin
    idtmin = np.argmin(np.abs(ti - tmin))
    idtmax = np.argmin(np.abs(ti - tmax))
    idmaxs = np.argmax(power[:, idtmin:idtmax], axis=1) + idtmin

    pickedArrivalWavelet = np.zeros_like(period)
    for i in range(power.shape[0]):
        pickedArrivalWavelet[i] = ti[idmaxs[i]]
    return pickedArrivalWavelet


pickedArrivalWavelet = pickArrivalWavelet(ti, power, periodsWT)
plt.plot(periodsWT, pickedArrivalWavelet)
plt.xlabel("period (s)")
plt.ylabel("picked time (s)")
plt.show()

Note that for period in the range 30 to 60s, there is energy arriving very early (between 500s and 1000s following the earthquake onset). We need to manually enforce that the picked time is greater than 1050s, to avoid picking wrong arrivals.   
Next, we compute the distance between source and receivers using obspy.

In [ ]:
def compute_distance_source_receiver(lons, lats, lonr, latr):
    dist_degree_sr = calc_dist(
        source_latitude_in_deg=lats,
        source_longitude_in_deg=lons,
        receiver_latitude_in_deg=latr,
        receiver_longitude_in_deg=lonr,
        radius_of_planet_in_km=6371,
        flattening_of_planet=0,
    )
    dist_km_sr = 6371 * dist_degree_sr * 2 * np.pi / 360.0
    return dist_degree_sr, dist_km_sr


dist_degree_sr, dist_km_sr = compute_distance_source_receiver(lons, lats, lonr, latr)
print(f"distance source-receivers: {dist_km_sr:.1f} km ({dist_degree_sr:.1f} deg)")

Now we plot the inferred dispersion curves and compare them with theoretical estimates.

In [ ]:
group_velocity_from_wavelet = dist_km_sr / pickedArrivalWavelet
plt.plot(myPeriods, dist_km_sr / pickedArrival, "x", label="this notebook")
plt.plot(periodsWT, group_velocity_from_wavelet, "o", label="this notebook, wavelet")
plt.xlabel("period (s)")
plt.ylabel("group velocity (km/s)")

surfaceType = "Love" if iZRT == 2 else "Rayleigh"
PREM_groupVel = np.loadtxt(f"PREM_groupVel{surfaceType}.dat", skiprows=1, delimiter=",")

plt.plot(PREM_groupVel[:, 0], PREM_groupVel[:, 1], label="PREM (G. Laske)")
plt.legend()
plt.show()

## Phase velocity

The frequency-dependent phase of a seismogram can be obtained by taking the Fourrier transform of it. It can be written as follows: 

\begin{equation*}
\Phi(\omega) = \omega t - k(\omega) x + \Phi_i(\omega) + 2n\pi = \omega t - \omega x / c(\omega)+ \Phi_i(\omega) + 2n\pi
\end{equation*}

Where $\omega t - \omega x / c(\omega)$ is the phase due to the propagation in space and time,  
$\Phi_i(\omega)$ depends on the source and  
$2n\pi$ reflects the periodicity of the phase.  
To compute the phase velocity we need to track the same wave at two receivers.  
Assuming that the receivers are at distance $x_1$ and $x_2$ from the source, and the wave passes at time $t_1$ and $t_2$ at each receivers, then the phase of both receivers would be:  

\begin{equation*}
\Phi_1(\omega) =   \omega t_1 - \omega x_1 / c(\omega)+ \Phi_i(\omega) + 2n\pi
\end{equation*}
and  
\begin{equation*}
\Phi_2(\omega) = \omega t_2 - \omega x_2 / c(\omega)+ \Phi_i(\omega) + 2m\pi
\end{equation*}

Then c can be obtained from the difference $\Phi_{21}(\omega)$ between $\Phi_2(\omega)$ and $\Phi_1(\omega)$:
\begin{equation*}
c(\omega) = \omega (x_2-x_1)/(\omega (t_2-t_1)+ 2(m-n)\pi- \Phi_{21}(\omega))
\end{equation*}
the term m-n is found empirically ensuring that the phase velocity is reasonable for long periods.  
source: Stein and Wysession.

In [ ]:
receiver2 = instaseis.Receiver(latitude=latr - 6, longitude=lonr)
st2 = db.get_seismograms(
    source=source, receiver=receiver2, components=["Z", "R", "T"], kind="displacement"
)
# convert to micro_m
for i in range(3):
    st2[i].data *= 1e6
st.plot()
st2.plot(show=False)

To facilitate comparison of the phase of the signals at the 2 stations, we align both signals by compensating for the across stations travel time of the wavepacket. We estimate it period-wise by interpolating the group slowness.

In [ ]:
dist_degree_sr_2, dist_km_sr_2 = compute_distance_source_receiver(lons, lats, lonr, latr - 6)
dist_km_src12 = dist_km_sr_2 - dist_km_sr
print(dist_km_src12)
f_group_slowness = interpolate.interp1d(periodsWT, 1.0 / group_velocity_from_wavelet)

The phase differences (and associated time shifts) between the aligned signals is associated with the difference between phase and group velocity.

In [ ]:
figall, axarr = plt.subplots(nper + 1, 1, figsize=(14, 18), dpi=160, sharex=False, sharey=False)

axarr[0].plot(ti, st[iZRT].data, label="no filter, r1")
axarr[0].plot(ti, st2[iZRT].data, label="no filter, r2")
axarr[0].legend()

pickedArrival = np.zeros((nper, 3))
pickedArrival[:, 0] = myPeriods

for i, T in enumerate(myPeriods):
    for k, sti in enumerate([st, st2]):
        st_temp = sti.copy()
        st_temp.filter("bandpass", freqmin=0.85 / T, freqmax=1.15 / T, corners=4, zerophase=True)
        # align signals given interpolated group velocity
        shift = 0 if k == 0 else f_group_slowness(T) * dist_km_src12
        axarr[i + 1].plot(
            ti - shift,
            st_temp[iZRT].data,
            label=f"T={T:.1f}s",
        )
        # Envelope of filtered data
        data_envelope = obspy.signal.filter.envelope(st_temp[iZRT].data)
        axarr[i + 1].plot(ti - shift, data_envelope.data)
        idmax = np.argmax(data_envelope.data)
        pickedArrival[i, k + 1] = st_temp[iZRT].times(reftime=t0)[idmax]
        axarr[i + 1].legend(loc="upper left")
        # center the plot on the max energy of the filtered signal
        tc = f_group_slowness(T) * dist_km_sr
        axarr[i + 1].set_xlim(tc - 8 * T, tc + 8 * T)

axarr[-1].set_xlabel("time (s)")
axarr[nper // 2].set_ylabel("displacement (um)")
plt.show()

Phase differences can be evaluated with cross-correlation (and is sensitive to phase shift). Below we provide a simple code estimating phase velocity.

In [ ]:
from obspy.signal.cross_correlation import correlate

myPeriods2 = np.logspace(np.log2(10.0 * dt), np.log2(duration / 15), 40, base=2)
phase_vel = np.zeros_like(myPeriods2)

nsamples = 1000
atshift = np.linspace(-nsamples, nsamples, 2 * nsamples + 1) * dt
cc_data = np.zeros((myPeriods2.shape[0], 2 * nsamples + 1))

# filter and cross-correlate both signals
x_filter = 0.25
for i, T in enumerate(myPeriods2):
    st_temp = st.copy()
    st_temp2 = st2.copy()
    for sti in [st_temp, st_temp2]:
        sti.filter(
            "bandpass",
            freqmin=(1.0 - x_filter) / T,
            freqmax=(1.0 + x_filter) / T,
            corners=4,
            zerophase=True,
        )
    cc = correlate(st_temp2[iZRT], st_temp[iZRT], nsamples)
    cc_data[i, :] = cc


figall, ax = plt.subplots()
Tc = 80
idT = np.argmin(np.abs(myPeriods2 - Tc))
plt.plot(atshift, cc_data[idT, :])
plt.ylabel("cross correlation amplitude")
plt.xlabel(" time shift(s)")
plt.title(f"cross-correlogram for central period {Tc}s")
plt.show()


# plot cross correlation array
figall, ax = plt.subplots()
X, Y = np.meshgrid(myPeriods2, atshift)
plt.pcolormesh(X, Y, cc_data.T, cmap="seismic")
plt.xlabel("period (s)")
plt.ylabel(" time shift(s)")
plt.title("cross-correlation array")
figall, ax = plt.subplots()

# Plot only possible values
Vmax = 7.0
id_realistic = np.where(atshift > dist_km_src12 / Vmax)[0]
atshift = atshift[id_realistic]
cc_data = cc_data[:, id_realistic]
V_phase = dist_km_src12 / atshift

# generate 2D plot
X, Y = np.meshgrid(myPeriods2, V_phase)
plt.pcolormesh(X, Y, cc_data.T, cmap="seismic")
# Plot reference data
surfaceType = "Love" if iZRT == 2 else "Rayleigh"
PREM_phaseVel = np.loadtxt(f"PREM_phaseVel{surfaceType}.dat", skiprows=1, delimiter=",")
plt.plot(PREM_phaseVel[:, 0], PREM_phaseVel[:, 1], label="PREM (G. Laske)")


plt.xlabel("period (s)")
plt.ylabel("phase velocity (km/s)")
plt.legend()
plt.show()